In [1]:
import os

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
# connect kafka with spark with simple word count example
# run on a terminal after starting kafka
# kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test
# kafka-console-producer --bootstrap-server localhost:9092 --topic test  

# kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic word-counts-1min
# kafka-console-consumer --bootstrap-server localhost:9092 --topic word-counts-1min --from-beginning

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()

22/03/14 21:08:54 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/14 21:08:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-afdd6689-2a05-4d49-89c5-8818ca54bad9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 624ms :: artifacts dl 10ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "test")\
  .option("group.id", "wordcount-group")\
  .load()

In [5]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
linesDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
linesDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [7]:
import pyspark.sql.functions as F

wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")).alias("word"), "timestamp" )
wordsDf.printSchema()
# apply 1 min window

wordCountsDf = wordsDf.groupBy("word", F.window(wordsDf.timestamp, "5 minutes")).count()
wordCountsDf.printSchema()

root
 |-- word: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

root
 |-- word: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- count: long (nullable = false)



In [8]:
echoOnconsole = wordCountsDf\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

22/03/14 21:09:56 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-46cc42cd-4afb-4e78-8314-fa11de4d3595. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [9]:
import pyspark.sql.functions as F

# convert all the columns into json
# * represent all columns ie word, count, 
# struct create a structure around word, count columns
# to json convert structure to column
# value is Kafka value part of message
wordCountsToKafkaDf = wordCountsDf\
                    .selectExpr("to_json(struct(*)) as value")

wordCountsToKafkaDf.printSchema()

root
 |-- value: string (nullable = true)



-------------------------------------------
Batch: 0
-------------------------------------------
+----+------+-----+
|word|window|count|
+----+------+-----+
+----+------+-----+



In [10]:
wordCountsToKafkaDf.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "word-counts-1min")\
                    .option("checkpointLocation", "file:///tmp/spark2")\
                    .start()
                    

-------------------------------------------
Batch: 1
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|10:40AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|10:40AM|{2022-03-14 21:10...|    2|
+-------+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    1|
|10:40AM|{2022-03-14 21:10...|    2|
+-------+--------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    1|
|10:43AM|{2022-03-14 21:10...|    1|
|10:40AM|{2022-03-14 21:10...|    2|
+-------+--------------------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    1|
|10:40AM|{2022-03-14 21:10...|    2|
|10:43AM|{2022-03-14 21:10...|    1|
|10:44AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+



...

-------------------------------------------
Batch: 8
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    2|
|10:40AM|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:15...|    2|
|10:43AM|{2022-03-14 21:10...|    1|
|10:44AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+



-------------------------------------------
Batch: 9
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    2|
|10:40AM|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:15...|    3|
|10:43AM|{2022-03-14 21:10...|    1|
|10:44AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+



-------------------------------------------
Batch: 10
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    2|
|10:40AM|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:15...|    4|
|10:43AM|{2022-03-14 21:10...|    1|
|10:44AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+



-------------------------------------------
Batch: 11
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:20...|    1|
|10:40AM|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:15...|    4|
|10:43AM|{2022-03-14 21:10...|    1|
|10:44AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+



-------------------------------------------
Batch: 12
-------------------------------------------
+-------+--------------------+-----+
|   word|              window|count|
+-------+--------------------+-----+
|Welcome|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:20...|    2|
|10:40AM|{2022-03-14 21:10...|    2|
|Welcome|{2022-03-14 21:15...|    4|
|10:43AM|{2022-03-14 21:10...|    1|
|10:44AM|{2022-03-14 21:10...|    1|
+-------+--------------------+-----+

